In [2]:
import data_preprocessor

abbreviations = data_preprocessor.load_json("abbreviations.json")

In [3]:
fname = 'data/khi_tickets_2022.csv'

df_complete = data_preprocessor.load_corpus(fname, pandas = True, header = True)

df = df_complete.drop(columns=['Title', 'Created', 'Close Time', 'Queue'], axis=1)

In [4]:
def pre_processing(address):
    standardized_address = data_preprocessor.lowercase_conversion(address)
    # standardized_address = data_preprocessor.remove_punctuation(standardized_address, True)
    standardized_address = data_preprocessor.standard_abbreviations_fix(standardized_address, abbreviations)
    # standardized_address = data_preprocessor.remove_extra_spaces(standardized_address, True)
    standardized_address = data_preprocessor.remove_extra_spaces(standardized_address, False)

    address_type = data_preprocessor.check_address_type(standardized_address)
    tokenized_address = data_preprocessor.standard_tokenization(standardized_address)

    # print(standardized_address)
    # print(address_type)
    # print(tokenized_address)

    return (standardized_address, address_type, tokenized_address)

In [5]:
test = df['Address'][0:10]

In [6]:
import random

def generate_random_numbers():
    random_numbers = []
    for _ in range(10):
        random_number = random.randint(0, 213874)
        random_numbers.append(random_number)
    return random_numbers


In [9]:
random_numbers = generate_random_numbers()

random_addresses = []

for i in random_numbers:
    random_addresses.append(df['Address'][i])

print(random_addresses)

[' House # 34/2  4th, Commercial Lane, Phase 4, Defence, Karachi ', ' House # A-419  , Street 22, Block 15, Gulistan-e-Johar, Karachi ', ' House # A-107, Street No 32, Sector-11-B, North Karachi, Karachi ', ' House # D-185 The Karachi Brightway Academy, Hamid H Farooqi Road, Block 2, PECHS, Karachi ', ' House # A 20, Sikandar Bakht Road, Block C, North Nazimabad, Karachi ', ' House # A-51 Ground Floor, Street No 13, PCSIR Society - Sector 24/A  , Gulzar-e-Hijri - Scheme-33, Karachi ', ' House # C-125/1, Street 14, Block 3 A, Gulistan-e-Johar, Karachi ', ' House # 206/D flat no 210 Singapore Center OFF, Tariq Road, Block 2, PECHS, Karachi ', ' House # 2B/2 27th Street Off, Khayaban e Shamsheer, Phase 5, Defence, Karachi ', ' House # R-19, Street  19, Block 10-A, Gulshan-e-Iqbal, Karachi ']


In [ ]:
def show_pre_processing(list_of_addresses):
    for ady in list_of_addresses:
        address_info = pre_processing(ady)
        print('Address: ', address_info[0])
        print('Type: ', address_info[1])
        print(' ')

show_pre_processing(random_addresses)

In [ ]:
show_pre_processing(test)

In [7]:
import pandas as pd

columns = ['Ticket #', 'Type', 'House #', 'Apartment #', 'Building #', 'Building Name', 'Street Number/Name', 'Area & Sub Area', 'Neighbourhood', 'City']

df1 = data_preprocessor.create_dataframe(columns)

# data = [['123456', 'Appartment', '123', 'A2', 'Building A', 'ABC Towers', 'Main Street', 'Central Area', 'ABC Neighborhood', 'City X']]

# data = {'Ticket #': [], 'Type': [], 'House #': [], 'Apartment #': [], 'Building #': [], 'Building Name': [], 'Street Number/Name': [], 'Area & Sub Area': [], 'Neighbourhood': [], 'City': []}

data = {'Ticket #': ['123456'], 'Type': ['Appartment'], 'House #': ['123'], 'Apartment #': ['A2'], 'Building #': ['Building A'], 'Building Name': ['ABC Towers'], 'Street Number/Name': ['Main Street'], 'Area & Sub Area': ['Central Area'], 'Neighbourhood': ['ABC Neighborhood'], 'City': ['City X']}

df2 = data_preprocessor.create_dataframe(columns, data, datacheck=True)

df1 = pd.concat([df1, df2], axis=0)


print(df1)



  Ticket #        Type House # Apartment #  Building # Building Name  \
0   123456  Appartment     123          A2  Building A    ABC Towers   

  Street Number/Name Area & Sub Area     Neighbourhood    City  
0        Main Street    Central Area  ABC Neighborhood  City X  


In [8]:
area_keywords = ['block', 'phase', 'scheme', 'sector']

def field_finder(field_name, tokenized_list):

    street_keywords = ['street', 'road', 'highway', 'lane', 'avenue', 'boulevard']
    house_keywords = ['house', 'house no', 'house number', 'house #', 'plot']
    apartment_keywords = ['flat', 'flat no', 'flat number', 'flat #', 'apartment', 'suite']
    floor_keywords = ['floor', 'fl', 'level']
    keywords = []
    
    field_name = field_name.lower()

    if field_name == 'street':
        keywords = street_keywords
    elif field_name == 'house':
        keywords = house_keywords
    elif field_name == 'apartment':
        keywords = apartment_keywords
    elif field_name == 'floor':
        keywords = floor_keywords

    for index, token in enumerate(tokenized_list):
        if any(keyword in token for keyword in keywords):
            return index
    
    return None

# house_index = field_finder('house', address)
# apartment_index = field_finder('apartment', address)
# street_index = field_finder('street', address)

# print('House Index:', house_index)
# print('Apartment Index:', apartment_index)
# print('Street Index:', street_index)


In [9]:
test2 = df[['Ticket#', 'Address']][0:20]

test2.head()


,Ticket#,Address
0,105207394681,"House # SC-4, Al Kareem Centre, Flat# 107, 1s..."
1,109247984771,"House # Plot# 28-C, Flat A-2, 1st Floor, Lane..."
2,101042042052,"Apartment/Suite# Flat 204, Building All Block..."
3,101087613080,"Apartment/Suite# BI3, Building All Blocks, Ib..."
4,107178824611,"Apartment/Suite# 26, Building Block B, Marhab..."


In [13]:
address_df = data_preprocessor.create_dataframe(columns)

In [14]:
# Hierarchal and Fields parsing: 

def parse(dataframe):

    global address_df

    list_of_addresses = dataframe['Address'].tolist()
    tickets = dataframe['Ticket#'].tolist()
    # counter = 0
    
    # for address in list_of_addresses:
    
    for index in range(len(dataframe)):
        ticketno = tickets[index]
        address = list_of_addresses[index]

        data = {'Ticket #': [], 'Type': [], 'House #': [], 'Apartment #': [], 'Building #': [], 'Building Name': [], 'Street Number/Name': [], 'Area & Sub Area': [], 'Neighbourhood': [], 'City': []}
        
        address_info = pre_processing(address)
        address_type = address_info[1]
        tokenized_address = address_info[2]

        data['Ticket #'].append(ticketno)
        # data['Ticket #'].append(tickets[counter])
        # counter+=1

        data['Type'].append(address_type)

        data['City'].append(tokenized_address.pop(-1))
        data['Neighbourhood'].append(tokenized_address.pop(-1))


        street_index = field_finder('street', tokenized_address)
        if street_index is not None:
            data['Street Number/Name'].append(tokenized_address.pop(street_index))
        else:
            data['Street Number/Name'].append('None')

        house_index = field_finder('house', tokenized_address)
        if house_index is not None:
            data['House #'].append(tokenized_address.pop(house_index))
        else:
            data['House #'].append('None')

        appartment_index = field_finder('apartment', tokenized_address)
        if appartment_index is not None:
            data['Apartment #'].append(tokenized_address.pop(appartment_index))
        else:
             data['Apartment #'].append('None')

        floor_index = field_finder('floor', tokenized_address)
        if floor_index is not None:
            if data['Apartment #'] == ['None']:
                data['Apartment #'] = [tokenized_address.pop(floor_index)]
            else:
                data['Apartment #'].append(tokenized_address.pop(floor_index))
                value_lst = data['Apartment #']
                joined_string = ' '.join(value_lst)
                data['Apartment #'] = [joined_string]
        # else:
        #      data['Apartment #'].append('None')


        for field in data:
            if len(data[field]) == 0:
                data[field].append('None')
                # data[field].append(-1)

        
        df_temp = data_preprocessor.create_dataframe(columns, data, datacheck=True)
        address_df = pd.concat([address_df, df_temp], axis=0)

    return address_df



parse(test2)

# parse(random_addresses)

# print(address_df)

address_df.head()


,Ticket #,Type,House #,Apartment #,Building #,Building Name,Street Number/Name,Area & Sub Area,Neighbourhood,City
0,105207394681,apartment,house # sc - 4,flat # 107 1st floor,None,None,faizan street,None,north nazimabad,karachi
0,109247984771,apartment,house # plot # 28 - c,flat a - 2 1st floor,None,None,lane 9,None,defence,karachi
0,101042042052,apartment,None,apartment / suite # flat 204,None,None,8th street,None,clifton,karachi
0,101087613080,apartment,None,apartment / suite # bi3,None,None,adam road,None,clifton,karachi
0,107178824611,apartment,None,apartment / suite # 26,None,None,akhtar masood road,None,north nazimabad,karachi


In [15]:
address_df.to_csv('data/data3.csv', index=False)
